In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append("../utils")

from utils import load_csv, save_data

In [7]:
import pandas as pd

def create_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()

    # EMA và SMA
    df['EMA_9'] = df['close'].ewm(span=9, adjust=False).mean().shift()
    df['SMA_5'] = df['close'].rolling(5).mean().shift()
    df['SMA_10'] = df['close'].rolling(10).mean().shift()
    df['SMA_15'] = df['close'].rolling(15).mean().shift()
    df['SMA_30'] = df['close'].rolling(30).mean().shift()

    # Tính RSI (14 ngày)
    delta = df['close'].diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)

    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()

    rs = avg_gain / avg_loss
    df['RSI_14'] = 100 - (100 / (1 + rs))

    # Tính MACD (12 EMA - 26 EMA)
    ema_12 = df['close'].ewm(span=12, adjust=False).mean()
    ema_26 = df['close'].ewm(span=26, adjust=False).mean()
    df['MACD'] = ema_12 - ema_26

    # Loại bỏ NaN (phát sinh do rolling/EMA)
    df.dropna(inplace=True)

    # Sắp xếp theo ngày tăng dần
    df = df.sort_values(by="date").reset_index(drop=True)
    return df


In [8]:
symbols = ["fpt", "hpg", "vnm"]
dfs = {}

In [9]:
for symbol in symbols:
    raw_path = f"../../data/clean/{symbol}.csv"
    df = load_csv(raw_path)
    df_features = create_features(df)
    dfs[symbol] = df_features
    print(f"✅ {symbol.upper()} - Rows: {df_features.shape[0]}")


2025-05-27 19:30:20,186 - INFO - ✅ Load CSV thành công: ../../data/clean/fpt.csv | shape = (4551, 6)
2025-05-27 19:30:20,264 - INFO - ✅ Load CSV thành công: ../../data/clean/hpg.csv | shape = (4348, 6)
2025-05-27 19:30:20,292 - INFO - ✅ Load CSV thành công: ../../data/clean/vnm.csv | shape = (4783, 6)


✅ FPT - Rows: 4521
✅ HPG - Rows: 4318
✅ VNM - Rows: 4753


In [10]:
for symbol in symbols:
    save_data(dfs[symbol], f"../../data/features/{symbol}_features.csv")
    print(f"💾 Saved: {symbol}_features.csv")

2025-05-27 19:30:21,169 - INFO - 💾 Đã lưu dữ liệu thành công: ../../data/features/fpt_features.csv
2025-05-27 19:30:21,219 - INFO - 💾 Đã lưu dữ liệu thành công: ../../data/features/hpg_features.csv
2025-05-27 19:30:21,268 - INFO - 💾 Đã lưu dữ liệu thành công: ../../data/features/vnm_features.csv


💾 Saved: fpt_features.csv
💾 Saved: hpg_features.csv
💾 Saved: vnm_features.csv
